In [1]:
from src.kr6 import KR6
import modern_robotics as mr
import numpy as np
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes
from src.dh import DHFrame
import src.tools as tools
import sympy as sp


In [21]:

# r1, r2, r3, r4, a1, a2 = 0, 0.400, 0.455, 10**(-3)*np.sqrt(420**2+35**2), 0.025, 0.035
# køddevinkel = np.arctan2(35, 420)

T_SB = np.array([[0,  1,   0, 0],
                 [0,  0,  -1, -(0.420+0.025+0.080)],
                 [-1, 0,   0, 0.89],
                 [0,  0,   0, 1]])

#print('forventer at P_W er: \n', tools.pointFromT(tools.makeT_SW(T_SB)))

theta = np.pi/180*np.array([0, -90, 90, 0, 0, 0])
theta1 = np.pi/180*np.array([0, -90, 90, 0, 0, 0])

kr6 = KR6(theta)

thetalists = np.around(kr6.aIK_thetaLists,2)

thetalists * 180/np.pi

array([[  -0.        ,   -8.59436693,  -80.21409132],
       [  -0.        ,  -89.95437384,   89.95437384],
       [ 179.90874767, -171.31438074,   83.07888029],
       [ 179.90874767,  -96.25690958,  -73.91155557]])

In [22]:
thetalist = tools.inverseKinematicsTheta456(thetalists[1], kr6.Tsb_poe,kr6._Slist,kr6._M)
thetalist*=180/np.pi
thetalist

array([ -0.        , -89.95437384,  89.95437384,   0.        ,
         0.        , 180.        ])

In [31]:
print(np.allclose(tools.IKinSpace_our(kr6._Slist,kr6._M,kr6.Tsb_poe,theta1,0.01,0.01)[0],thetalist,0.1))
tools.IKinSpace_our(kr6._Slist,kr6._M,kr6.Tsb_poe,theta1,0.01,0.01)[0]*180/np.pi


False


array([  0., -90.,  90.,   0.,   0.,   0.])

In [5]:

# th4, th5, th6 = sp.symbols("theta_4 theta_5 theta_6")

# def rotX_sp(th):
#     x = sp.Matrix([[1, 0, 0],
#                [0, sp.cos(th), -sp.sin(th)],
#                [0, sp.sin(th), sp.cos(th)]])
#     return x

# def rotZ_sp(th):
#     z = sp.Matrix([[sp.cos(th), -sp.sin(th), 0],
#                [sp.sin(th), sp.cos(th), 0],
#                [0, 0, 0]])
#     return z

# def rotxyx(th4,th5,th6):
    th4 = -th4
    th6 = -th6
    # c4, s4 = sp.cos(th4), sp.sin(th4)
    # c5, s5 = sp.cos(th5), sp.sin(th5)
    # c6, s6 = sp.cos(th6), sp.sin(th6)

    # x = sp.Matrix([[1, 0, 0],
    #            [0, sp.cos(th4), -sp.sin(th4)],
    #            [0, sp.sin(th4), sp.cos(th4)]])

    # y = sp.Matrix([[sp.cos(th5), 0, sp.sin(th5)],
    #            [0, 0, 0],
    #            [-sp.sin(th5), 0, sp.cos(th5)]])

    # x2 = sp.Matrix([[1, 0, 0],
    #            [0, sp.cos(th6), -sp.sin(th6)],
    #            [0, sp.sin(th6), sp.cos(th6)]])

    # z = sp.Matrix([[sp.cos(th6), -sp.sin(th6), 0],
    #            [sp.sin(th6), sp.cos(th6), 0],
    #            [0, 0, 0]])
    
    R_45 = sp.Matrix([[sp.cos(th4), -sp.sin(th4), 0],
               [sp.sin(th4), sp.cos(th4), 0],
               [0, 0, 0]]) @ sp.Matrix([[1, 0, 0],
               [0, sp.cos(-PI/2), -sp.sin(-PI/2)],
               [0, sp.sin(-PI/2), sp.cos(-PI/2)]])

    R_56 = sp.Matrix([[sp.cos(th5), -sp.sin(th5), 0],
               [sp.sin(th5), sp.cos(th5), 0],
               [0, 0, 0]]) @ sp.Matrix([[1, 0, 0],
               [0, sp.cos(PI/2), -sp.sin(PI/2)],
               [0, sp.sin(PI/2), sp.cos(PI/2)]])


    R_6B = sp.Matrix([[sp.cos(th6), -sp.sin(th6), 0],
               [sp.sin(th6), sp.cos(th6), 0],
               [0, 0, 0]]) @ sp.Matrix([[1, 0, 0],
               [0, sp.cos(PI), -sp.sin(PI)],
               [0, sp.sin(PI), sp.cos(PI)]])
    # rot_youtube = sp.Matrix([[c4*s5*c6 - s4*s6, -c4*s5*s6 - s4*c6, -c4*c5],
    #                [s4*s5*c6 + c4*s6, -s4*s5*s6 + c4*c6, -s4*c5],
    #                [c5*c6, -c5*s6, s5]])

    # x = sp.Matrix([[, , ],
    #                [, , c4],
    #                [, , ]])
    # x = sp.Matrix([[1, 0, 0],
    #                [0, 0, 0],
    #                [0, 0, 0]])
    # x = sp.Matrix([[1, 0, 0],
    #                [0, 0, 0],
    #                [0, 0, 0]])
    return R_45,R_56,R_6B

# print(np.around(R_4B,2))
# print(np.arccos(-1))
# PI = np.pi
# R_45 = rotZ_sp(th4) @ rotX_sp(-PI/2)
# R_56 = rotZ_sp(th5) @ rotX_sp(PI/2) 
# R_6B = rotZ_sp(th6) @ rotX_sp(PI)
# R_45.evalf(1)*R_56.evalf(1)*R_6B.evalf(1)


IndentationError: unexpected indent (Temp/ipykernel_16636/4127285732.py, line 16)

In [ ]:
# config_fasit = np.array([0,-np.pi/4, np.pi/6, 0,0,0])
# config2 = np.array([0, -np.pi/2, np.pi/2, 0, 0, 0])
# config3 = np.array([0, np.pi/2, 0, 0, 0, 0])





# # kr6.add_FK_to_viewer(viewer)

# # print(f"Number of iteration: {kr6.IK[3]}")

# # kr6.add_nIK_to_viewer(viewer)
# viewer = Viewer(background="white")
# viewer.add(Grid())
# KR6(config_fasit).add_FK_to_viewer(viewer)
